In [33]:
import numpy as np
import os as os
import pickle as pickle
import tensorflow as tf
tfk = tf.keras

Generate points of our CY manifold X (in this case X - fermat quintic) 

In [34]:
from cymetric.pointgen.pointgen import PointGenerator

monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

pg = PointGenerator(monomials, coefficients, kmoduli, ambient)

Creating dataset for neural network

In [35]:
dirname = 'fermat_points'
n_p = 100000 # we will generate 100k points 
kappa = pg.prepare_dataset(n_p, dirname) # volume of CY over

pointgen:INFO:Vol_k: 5.000000000000001, Vol_cy: 284.26406800526263.


In [36]:
data = np.load(os.path.join(dirname, 'dataset.npz'))

Information about basis, coefficients, etc. of monomials and their derivatives + other relevant info that we will pass to our model

In [37]:
pg.prepare_basis(dirname, kappa=kappa)

0

In [38]:
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
from cymetric.models.tfhelper import prepare_tf_basis, train_model
BASIS = prepare_tf_basis(BASIS)

In [39]:
kappa = np.real(BASIS['KAPPA'].numpy());
kappa

0.017589279

Ricci measure shows how close to zero the ricci scalar is 

In [40]:
from cymetric.models.callbacks import RicciCallback
rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
cb_list = [rcb]


Hyperparameteres

In [41]:
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 50
bSizes = [64, 50000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5
n_out = nfold**2

Neural net

In [42]:
nn = tf.keras.Sequential()
nn.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn.add(tfk.layers.Dense(nHidden, activation=act))
nn.add(tfk.layers.Dense(n_out, use_bias=False))

In [44]:
from cymetric.models.tfmodels import MultFSModel
nnmetric = MultFSModel(nn, BASIS, alpha=alpha) 

In [45]:
from cymetric.models.metrics import SigmaLoss, KaehlerLoss, TransitionLoss, VolkLoss, RicciLoss, TotalLoss

cmetrics = [TotalLoss(), SigmaLoss(), KaehlerLoss(), TransitionLoss(), VolkLoss(), RicciLoss()]
opt = tfk.optimizers.Adam()

In [46]:
nnmetric, training_history = train_model(nnmetric, data, optimizer=opt, epochs=nEpochs, batch_sizes=[64, 50000], 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)


Epoch  1/50


C:\Users\Salih\anaconda3\envs\cymetric\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/GatherV2_5_grad/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradients/GatherV2_5_grad/Reshape:0", shape=(None,), dtype=complex64), dense_shape=Tensor("gradients/GatherV2_5_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\Salih\anaconda3\envs\cymetric\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/GatherV2_4_grad/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradients/GatherV2_4_grad/Reshape:0", shape=(None,), dtype=complex64), dense_shape=Tensor("gradients/GatherV2_4_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amo

2/2 [==============================] - 18s 16s/step - loss: 0.1512 - sigma_loss: 0.1177 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0335 - ricci_loss: 0.0000e+00 - ricci_val: 1.2342

Epoch  2/50
2/2 [==============================] - 2s 1s/step - loss: 0.3769 - sigma_loss: 0.0946 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.2823 - ricci_loss: 0.0000e+00 - ricci_val: 1.0526

Epoch  3/50
2/2 [==============================] - 2s 1s/step - loss: 0.1900 - sigma_loss: 0.0908 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0992 - ricci_loss: 0.0000e+00 - ricci_val: 1.0955

Epoch  4/50
2/2 [==============================] - 2s 1s/step - loss: 0.3339 - sigma_loss: 0.0893 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.2446 - ricci_loss: 0.0000e+00 - ricci_val: 1.1043

Epoch  5/50
2/2 [==============================] - 2s 1s/step - loss: 0.1812 - sigma_loss: 0.0846 - kaehler_loss: 0.0000e+00 -


Epoch 14/50
2/2 [==============================] - 3s 1s/step - loss: 0.2514 - sigma_loss: 0.0788 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1726 - ricci_loss: 0.0000e+00 - ricci_val: 0.9295

Epoch 15/50
2/2 [==============================] - 3s 1s/step - loss: 0.2550 - sigma_loss: 0.0766 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1784 - ricci_loss: 0.0000e+00 - ricci_val: 0.9305

Epoch 16/50
2/2 [==============================] - 3s 1s/step - loss: 0.2424 - sigma_loss: 0.0740 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1684 - ricci_loss: 0.0000e+00 - ricci_val: 0.8721

Epoch 17/50
2/2 [==============================] - 3s 1s/step - loss: 0.1836 - sigma_loss: 0.0774 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1061 - ricci_loss: 0.0000e+00 - ricci_val: 0.9285

Epoch 18/50
2/2 [==============================] - 3s 1s/step - loss: 0.1706 - sigma_loss: 0.0747 - kaehler_loss: 0


Epoch 27/50
2/2 [==============================] - 3s 1s/step - loss: 0.1711 - sigma_loss: 0.0691 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1021 - ricci_loss: 0.0000e+00 - ricci_val: 0.7572

Epoch 28/50
2/2 [==============================] - 3s 1s/step - loss: 0.1853 - sigma_loss: 0.0671 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1182 - ricci_loss: 0.0000e+00 - ricci_val: 0.6767

Epoch 29/50
2/2 [==============================] - 3s 1s/step - loss: 0.1097 - sigma_loss: 0.0624 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0474 - ricci_loss: 0.0000e+00 - ricci_val: 0.6813

Epoch 30/50
2/2 [==============================] - 3s 1s/step - loss: 0.1707 - sigma_loss: 0.0507 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.1200 - ricci_loss: 0.0000e+00 - ricci_val: 0.5678

Epoch 31/50
2/2 [==============================] - 3s 1s/step - loss: 0.1945 - sigma_loss: 0.0494 - kaehler_loss: 0


Epoch 40/50
2/2 [==============================] - 3s 1s/step - loss: 0.0927 - sigma_loss: 0.0209 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0718 - ricci_loss: 0.0000e+00 - ricci_val: 0.1510

Epoch 41/50
2/2 [==============================] - 3s 1s/step - loss: 0.0722 - sigma_loss: 0.0224 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0499 - ricci_loss: 0.0000e+00 - ricci_val: 0.1542

Epoch 42/50
2/2 [==============================] - 3s 1s/step - loss: 0.1040 - sigma_loss: 0.0177 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0862 - ricci_loss: 0.0000e+00 - ricci_val: 0.1495

Epoch 43/50
2/2 [==============================] - 3s 1s/step - loss: 0.1136 - sigma_loss: 0.0185 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0951 - ricci_loss: 0.0000e+00 - ricci_val: 0.1329

Epoch 44/50
2/2 [==============================] - 3s 1s/step - loss: 0.1272 - sigma_loss: 0.0172 - kaehler_loss: 0